Deskripsi Data -
- Battery_power - Total energi yang dapat disimpan oleh baterai dalam satu kali pengisian, diukur dalam mAh
- Blue - Memiliki bluetooth atau tidak
- Clock_speed - Kecepatan di mana mikroprosesor menjalankan instruksi
- Dual_sim - Mendukung dual sim atau tidak
- Fc - Resolusi kamera depan dalam mega pixel
- Four_g - Mendukung 4G atau tidak
- Int_memory - Memori Internal dalam Gigabytes
- M_dep - Kedalaman ponsel dalam cm
- Mobile_wt - Berat ponsel
- N_cores - Jumlah core processor
- Pc - Resolusi kamera utama dalam mega pixel
- Px_height - Tinggi resolusi pixel
- Px_width - Lebar resolusi pixel
- Ram - Memori Akses Acak dalam Mega
- Touch_screen - Memiliki layar sentuh atau tidak
- Wifi - Memiliki wifi atau tidak
- Sc_h - Tinggi layar ponsel dalam cm
- Sc_w - Lebar layar ponsel dalam cm
- Talk_time - Waktu penggunaan ponsel dengan sekali pengisian baterai
- Three_g - Mendukung 3G atau tidak
- Wifi - Memiliki wifi atau tidak
- Price_range - Variabel target dengan nilai 0(low cost), 1(medium cost), 2(high cost) and 3(very high cost).

In [12]:
import pandas as pd
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



In [13]:
# Memuat data
data = pd.read_csv('data/dataset.csv')

data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [14]:
# Menampilkan fitur atau kolom yang ada
data.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [15]:
data.shape

(2000, 21)

Dataset ini ada 2000 baris dan 21 kolom. Kolom terakhir adalah nilai target, nilai yang ingin kita prediksi. Dalam proyek ini, kita akan memprediksi rentang harga ponsel menggunakan daftar spesifikasi ponsel dengan menerapkan beberapa algoritma machine learning.

In [16]:
#cek missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

Tidak ada missing values di dataset

In [17]:
print(data.groupby('price_range').size())
#data set is balanced 

price_range
0    500
1    500
2    500
3    500
dtype: int64


kode data_table.corr()['price_range'] digunakan untuk menghitung korelasi antara fitur-fitur dalam data_table dengan kolom 'price_range'. Korelasi adalah ukuran statistik yang menggambarkan hubungan linier antara dua variabel. Dalam konteks ini, kita ingin mengetahui seberapa kuat hubungan antara setiap fitur dengan 'price_range' (rentang harga). Semakin mendekati 1 atau -1, semakin kuat hubungan linier antara fitur tersebut dengan 'price_range'. Jika mendekati 0, maka hubungannya cenderung lemah atau tidak ada hubungan linier yang signifikan antara fitur tersebut dengan 'price_range'.

In [18]:
data.corr()['price_range']

battery_power    0.200723
blue             0.020573
clock_speed     -0.006606
dual_sim         0.017444
fc               0.021998
four_g           0.014772
int_memory       0.044435
m_dep            0.000853
mobile_wt       -0.030302
n_cores          0.004399
pc               0.033599
px_height        0.148858
px_width         0.165818
ram              0.917046
sc_h             0.022986
sc_w             0.038711
talk_time        0.021859
three_g          0.023611
touch_screen    -0.030411
wifi             0.018785
price_range      1.000000
Name: price_range, dtype: float64

Disini saya bagi datasetnya menjadi independent kolom dan target kolom

In [19]:
X = data.iloc[:,0:20] #independenet columns 
Y = data.iloc[:,-1] # target column"

Kode dibawah ini digunakan untuk melakukan seleksi fitur pada data menggunakan metode chi-square. Seleksi fitur adalah proses memilih subset fitur yang paling informatif atau memiliki hubungan yang kuat dengan target yang ingin diprediksi. Metode chi-square digunakan untuk mengukur hubungan antara variabel kategorikal, seperti fitur-fitur dalam dataset, dengan variabel target. Nanti akan menghasilkan tabel yang menunjukkan 15 fitur terbaik berdasarkan skor chi-square


In [20]:
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 
#get the k optimal value that we just looped just now 
bestfeatures = SelectKBest(score_func = chi2,k=17) 
fit = bestfeatures.fit(X,Y) 
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns) 
# making sort of a datatable to visualize the values 
score_table = pd.concat([dfcolumns,dfscores],axis = 1)
# name the table column
score_table.columns = ['Specifications','Score'] 
# print 15 best features 
print(score_table.nlargest(15,'Score'))

   Specifications          Score
13            ram  931267.519053
11      px_height   17363.569536
0   battery_power   14129.866576
12       px_width    9810.586750
8       mobile_wt      95.972863
6      int_memory      89.839124
15           sc_w      16.480319
16      talk_time      13.236400
4              fc      10.135166
14           sc_h       9.614878
10             pc       9.186054
9         n_cores       9.097556
18   touch_screen       1.928429
5          four_g       1.521572
7           m_dep       0.745820


Kemudian saya hapus fitur fitur yang tidak relevan. Karena saya menghapus three_g, yang terbaik adalah menghapus four_g juga karena dari heatmap ada beberapa korelasi

In [21]:
del data['three_g']
del data['wifi']
del data['dual_sim'] 
del data['clock_speed'] 
del data['blue'] 

# menghapus four_g juga karena ada beberapa korelasi
del data['four_g']

## **Training Models**

In [22]:
# mendifinisikan X dan Y, lalu ubah ke numpy
Y = data['price_range'].to_numpy()
del data['price_range']
X = data.to_numpy()

In [23]:
# Test options and evaluation metric
validation_size = 0.30
seed = 7
scoring = 'accuracy'
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [24]:
### Normal Default configurations of models
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier


### 1. SVM

#### Building Support Vector Machine 

In [25]:
svm = SVC()
svm_model = svm.fit(X_train,Y_train)
svm_predict = svm_model.predict(X_validation)
accuracy_svm = accuracy_score(Y_validation,svm_predict)
print("Accuracy of the svm model: ", accuracy_svm * 100)
joblib.dump(svm_model,'models/svm.pkl')


Accuracy of the svm model:  95.0


['models/svm.pkl']

### 2. Random Forest Model

#### Building Default Random Forest Model

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
MODEL2 = rf.fit(X_train,Y_train)
rf_predict = MODEL2.predict(X_validation)
accuracy_rf = accuracy_score(Y_validation,rf_predict)
print("Accuracy of Random Forest Model:",accuracy_rf)
joblib.dump(MODEL2,'models/randomForest.pkl')

Accuracy of Random Forest Model: 0.8766666666666667


['models/randomForest.pkl']

### 3. Decision Tree Model

In [27]:
from sklearn.tree import DecisionTreeClassifier
dt_classfier = DecisionTreeClassifier()
model_dt = dt_classfier.fit(X_train,Y_train)
dt_classi_predict = model_dt.predict(X_validation)
accuravy_DT = accuracy_score(Y_validation,dt_classi_predict)
print("Accuravy of the decision tree classifier model: ", accuravy_DT * 100)
joblib.dump(model_dt,'models/decisionTreeClassifier.pkl')

Accuravy of the decision tree classifier model:  82.66666666666667


['models/decisionTreeClassifier.pkl']